# Merge all the diffrent data to one big data frame

In [ ]:
# Data handling and plotting
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import parser  # More robust date parsing

# MongoDB
from pymongo import MongoClient
from bson import ObjectId


In [ ]:
heart_rate = pd.read_csv("csv/Heart_Rate.csv")
print(heart_rate.head())

hrv = pd.read_csv("csv/HRV.csv")
print(hrv.head())

mood = pd.read_csv("csv/Mood.csv")
print(mood.head())

sleep = pd.read_csv("csv/sleep.csv")
print(sleep.head())

steps = pd.read_csv("csv/steps.csv")
print(steps.head())

resting_heart_rate = pd.read_csv("csv/resting_heart_rate.csv")
print(resting_heart_rate.head())

In [ ]:
merged = heart_rate.merge(hrv,on='date',how='outer').merge(mood,on='date',how='outer').merge(sleep,on='date',how='outer').merge(steps,on='date',how='outer').merge(resting_heart_rate,on='date',how='outer')

In [ ]:
merged.describe()